# brighton and hove busines properties 

thanks to [adam dennett](https://adamdennett.co.uk), we have a dataset on the properties of local businesses ([see here](https://www.brighton-hove.gov.uk/business-and-trade/business-rates-information-city-businesses)).

after fetching the most recent download of the local business data (true as of 2025-04) and placing in the data folder, this code is for reading, parsing, claening the data in that excel sheet and saving them as a table in the `duckdb` database.


In [1]:
import datetime
import duckdb
import numpy as np
from openpyxl import load_workbook
import pandas as pd
import requests

In [2]:
DATA_PATH = '../data/'
PATH_TO_DB = DATA_PATH+'creatives.duckdb'
PATH_TO_XCL = DATA_PATH+'excel/'                # input
FILENAMES = [
    'brighton_and_hove_business_information_2025_04.xlsx'
]

# file selection

in this case there is only one relevant file to read in, parse, and ingest. 

In [3]:
filename = FILENAMES[0]
tablename = filename.split('.')[0]
filepath = PATH_TO_XCL + FILENAMES[0]
print(tablename, filepath)

brighton_and_hove_business_information_2025_04 ../data/excel/brighton_and_hove_business_information_2025_04.xlsx


In [4]:
wb = load_workbook(filename = filepath)
print('excel workbook', FILENAMES[0], 'contains the following sheet(s)', wb.sheetnames)

excel workbook brighton_and_hove_business_information_2025_04.xlsx contains the following sheet(s) ['bhn013_(NNDR_FOI_Extract)_Creat']


the workbooks contains a single sheet, with a somewhat obscure name.

In [5]:
sheet = wb.active
sheet.title

'bhn013_(NNDR_FOI_Extract)_Creat'

In [6]:
sheet = wb.active
print('determine the header row for sheet', sheet.title)
# assume row 1 is header, and that column count < 26^2
cell_range = sheet['A1':'ZZ1'] 
header_row = [[cell.value for cell in component if cell.value] for component in cell_range][0]
num_columns = len(header_row)
print(num_columns, 'columns:', header_row)

determine the header row for sheet bhn013_(NNDR_FOI_Extract)_Creat
43 columns: ['Property_Ref', 'VOA_Ref', 'UPRN', 'BID_Zone', 'Name', 'Trading_As', 'PropDescr', 'Property1', 'Property2', 'Property3', 'Property4', 'Property5', 'Property6', '2017_RV', '2023_RV', 'Acc_Start', 'Acc_End', 'Base_Chg', 'Transitional_Relief-Surcharge', 'Part_Occ', 'Charity_Relief', 'SBRR', 'Other_Relief', 'Mandatory_Charity', 'Mand_Char_Start', 'Discretionary_Charity', 'Disc_Char_Start', 'Top-up_Charity', 'Top-up_Char_Start', 'Small_Business_Rate_Relief', 'SBRR_Start', 'Supporting_Small_Businesses_Rel', 'SSB_Start', 'Retail_Discount', 'Ret_Discount_Start', 'Nursery_Relief', 'Nursery_Rel_Start', 'Public_Loo_Relief', 'Loo_Relief_Start', 'Exemption', 'Exemption_Start', 'Empty_Relief', 'Empty_Start']


note that some of the column names are inappropriate for a database table


# prepare a dataframe
next we set up a new dataframe to collect the table data into. all columns will be of data type str (pandas object) to begin with. then we will recast them. 

then we will read the excel file, one row at a time and populate the dataframe rows. this is a slow step and might take some time.

In [7]:
%%time
df = pd.DataFrame(columns=header_row) 
# append table, one row at a time
for row in sheet.rows:
    data_row = [element.value for element in row]
    df.loc[len(df)] = data_row[:num_columns]

CPU times: user 1min 52s, sys: 8.34 s, total: 2min
Wall time: 2min 1s


since we added the header row to the dataframe as a data row, we might need to trip the dataframe's top row:

In [8]:
if df['Property_Ref'][0]=='Property_Ref':
    print('need to trim top (header) row off the dataframe')
    df.drop(index=df.index[0], axis=0, inplace=True)

need to trim top (header) row off the dataframe


# check contents of the dataframe
a quick look into the dataframe's columns, types, and null occurrances.

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11164 entries, 1 to 11164
Data columns (total 43 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   Property_Ref                     11164 non-null  object
 1   VOA_Ref                          11164 non-null  object
 2   UPRN                             10752 non-null  object
 3   BID_Zone                         457 non-null    object
 4   Name                             7845 non-null   object
 5   Trading_As                       90 non-null     object
 6   PropDescr                        11164 non-null  object
 7   Property1                        11164 non-null  object
 8   Property2                        11164 non-null  object
 9   Property3                        6867 non-null   object
 10  Property4                        1823 non-null   object
 11  Property5                        32 non-null     object
 12  Property6                        1116

# data cleaning and data type casting

In [10]:
df = \
(
    df
    .assign(base_charge        = pd.to_numeric(df['Base_Chg']))  # float, monitary values
    .assign(uprn = pd.to_numeric(
        df['UPRN']
            .replace([np.inf, -np.inf], np.nan)
            .fillna('0'),
        downcast='integer',
        errors='coerce')
    )                                                            # integer id number
    .assign(rv_2017 = pd.to_numeric(
        df['2017_RV']
            .replace([np.inf, -np.inf], np.nan)
            .fillna('0'),
        downcast='integer',
        errors='coerce')
    )
    .assign(rv_2023 = pd.to_numeric(
        df['2023_RV']
            .replace([np.inf, -np.inf], np.nan)
            .fillna('0'),
        downcast='integer',
        errors='coerce')
    )
    .assign(charity_relief     = pd.to_numeric(
        df['Charity_Relief']
            .replace([np.inf,-np.inf], np.nan)
            .fillna('0'),
        downcast='integer',
        errors='coerce')
    )
    .assign(sbrr = pd.to_numeric(
        df['SBRR']
            .replace([np.inf,-np.inf], np.nan)
            .fillna('0'),
        downcast='integer',
        errors='coerce')
    )
    .assign(
        transitional_relief_surcharge=pd.to_numeric(
            df['Transitional_Relief-Surcharge'])
    )
    # datetime fields
    .assign(acc_start          = pd.to_datetime(df['Acc_Start']))          # 
    .assign(acc_end            = pd.to_datetime(df['Acc_End']))            # 
    .assign(sbrr_start         = pd.to_datetime(df['SBRR_Start']))         # 
    .assign(ssb_start          = pd.to_datetime(df['SSB_Start']))          # 
    .assign(mand_char_start    = pd.to_datetime(df['Mand_Char_Start']))    # 
    .assign(disc_char_start    = pd.to_datetime(df['Disc_Char_Start']))    # 
    .assign(ret_discount_start = pd.to_datetime(df['Ret_Discount_Start'])) # 
    .assign(top_up_char_start  = pd.to_datetime(df['Top-up_Char_Start']))  # 
    .assign(nursery_rel_start  = pd.to_datetime(df['Nursery_Rel_Start']))  # 
    .assign(loo_relief_start   = pd.to_datetime(df['Loo_Relief_Start']))   # 
    .assign(exemption_start    = pd.to_datetime(df['Exemption_Start']))    # 
    .assign(empty_start        = pd.to_datetime(df['Empty_Start']))        # 
    # clean up. drop useless and redundant fields.
    .drop(
        columns=[
            'UPRN',
            '2017_RV',
            '2023_RV',
            'Mand_Char_Start',
            'Acc_Start',
            'Acc_End',
            'SBRR_Start',
            'SSB_Start',
            'Ret_Discount_Start',
            'Exemption_Start',
            'Empty_Start',
            'Base_Chg',
            'Charity_Relief',
            'SBRR',
            'Transitional_Relief-Surcharge',            
        ]
    )
    # the rest of the columns: ensure their names are database friendly:
    .rename(columns=
        {
            'Property_Ref':'property_ref', 
            'VOA_Ref':'voa_ref',
            'BID_Zone':'bid_zone',
            'Name':'name',
            'Trading_As':'trading_as',
            'PropDescr':'propdescr',
            'Property1':'property_1',
            'Property2':'property_2',
            'Property3':'property_3',
            'Property4':'property_4',
            'Property5':'property_5',
            'Property6':'property_6',
            'Part_Occ':'part_occ',
            'Other_Relief':'other_relief',
            'Mandatory_Charity':'mandatory_charity',
            'Discretionary_Charity':'discretionary_charity',
            'Top-up_Charity':'top_up_charity',
            'Small_Business_Rate_Relief':'small_business_rate_relief',
            'Supporting_Small_Businesses_Rel':'supporting_small_business_rel',
            'Retail_Discount':'retail_discount',
            'Nursery_Relief':'nursery_relief',
            'Public_Loo_Relief':'public_loo_relief',
            'Exemption':'exemption',
            'Empty_Relief':'empty_relief'
        }
    )
)

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11164 entries, 1 to 11164
Data columns (total 47 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   property_ref                   11164 non-null  object        
 1   voa_ref                        11164 non-null  object        
 2   bid_zone                       457 non-null    object        
 3   name                           7845 non-null   object        
 4   trading_as                     90 non-null     object        
 5   propdescr                      11164 non-null  object        
 6   property_1                     11164 non-null  object        
 7   property_2                     11164 non-null  object        
 8   property_3                     6867 non-null   object        
 9   property_4                     1823 non-null   object        
 10  property_5                     32 non-null     object        
 11  property_6          

In [12]:
with duckdb.connect(database=PATH_TO_DB, read_only=False) as con:
    con.sql(f"DROP TABLE IF EXISTS {tablename};")
    con.sql(f"CREATE TABLE {tablename} AS SELECT * FROM df;")
    con.sql(f"SELECT COUNT(*) FROM {tablename};")